In [14]:
from datasets import load_dataset

ds = load_dataset("futurehouse/lab-bench", "LitQA2")

In [15]:
ds["train"].to_csv("train.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 41.63ba/s]


175783

In [22]:
import pandas as pd
df = pd.read_csv("train.csv")
df.columns

Index(['id', 'question', 'ideal', 'distractors', 'canary', 'tag', 'version',
       'sources', 'is_opensource', 'subtask', 'key-passage'],
      dtype='object')

In [ ]:
import requests
def fetch_pdf_link(doi, email):
    """
    Fetches the PDF link for a given DOI using the Unpaywall API.
    """
    url = f"https://api.unpaywall.org/v2/{doi}?email={email}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pdf_url = data.get('best_oa_location', {}).get('url_for_pdf', None)
        return pdf_url
    else:
        return None

# Add your email (required for Unpaywall API)
email = "wzehrakorkusuz@gmail.com"

# Fetch PDF links for all DOIs
df['pdf_url'] = df['sources'].apply(lambda x: fetch_pdf_link(x, email))

In [ ]:
import os

def download_pdf(pdf_url, output_dir, article_id):
    """
    Downloads the PDF file from the given URL.
    """
    if not pdf_url:
        print(f"No PDF available for {article_id}")
        return
    
    try:
        response = requests.get(pdf_url, stream=True)
        if response.status_code == 200:
            # Save the PDF to the specified directory
            os.makedirs(output_dir, exist_ok=True)
            file_path = os.path.join(output_dir, f"{article_id}.pdf")
            with open(file_path, 'wb') as pdf_file:
                pdf_file.write(response.content)
            print(f"Downloaded: {file_path}")
        else:
            print(f"Failed to download PDF for {article_id}: {response.status_code}")
    except Exception as e:
        print(f"Error downloading PDF for {article_id}: {e}")

# Directory to save PDFs
output_dir = "pdfs"

# Download PDFs
df.apply(lambda row: download_pdf(row['pdf_url'], output_dir, row['id']), axis=1)
